In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
import MCF_INV as inv
import scipy.optimize as sco
import plotly.graph_objects as go

<font color='DarkViolet ' style="font-size:25px"><b>CAPM and Factor Investing</b></font>

In [2]:
def erCAPM(ret,market,rf,f=252):
    """Computes expected returns by assuiming that: (1) CAPM holds and (2) arithmetic mean is good estimate of expected value.
    - ret: dataset of returns
    - market: vector of market returns
    - rf: vector of risk-free rate
    - f: optional, frequency at which data is collected (by default it is 252, i.e. dailiy data is assumed)"""
    RmRf=market-rf
    if isinstance(ret,pd.DataFrame):
        b=np.array([sm.OLS(ret[r]-rf, RmRf).fit().params['x1'] for r in ret.columns])
        return pd.Series(f*(rf.mean()+b*RmRf.mean()),index=ret.columns)
    else:
        b=sm.OLS(ret-rf, RmRf).fit().params['x1']
        return f*(rf.mean()+b*RmRf.mean())

In [3]:
def erPCA(ret,rf,f=252,n=None):
    """Computes expected returns based on PCA factor model assuming that arithmetic mean is correct estimate of expected
    returns. Parameters are:
    - ret: dataset of returns
    - rf: vector of risk-free rates
    - f: optional, frequency at which data is collected (by default daily frequency is assumed)
    - n: optional, number of components to be extracted (by default algorithm extract components by Kaisers criterion)"""
    if n!=None and ret.shape[1]<n:
        return print('Wrong input. Desired number of components (n) has to be smaller than or equal to number of stocks')
    else:
        l,evec=np.linalg.eig(ret.cov()*f)
        a=(evec[:,:np.count_nonzero(l>l.mean())] if n==None else evec[:,:n])
        rets_pca=ret@a
        errf,erpca=rf.mean(),rets_pca.mean()
        return [f*(errf+sm.OLS(ret[c]-rf,rets_pca).fit().params@erpca) for c in ret.columns]

<font color='DarkViolet ' style="font-size:15px"><b>Energy and Technology</b></font>

In [4]:
stocks = pd.read_csv("Energy and Technology.csv",header=0)
energy = stocks[stocks["Industry"]=="Energy"]["Ticker"].to_list()
tech = stocks[stocks["Industry"]=="Technology"]["Ticker"].to_list()

In [5]:
prices_energy = yf.download(energy,  start='2022-01-01', end ='2024-04-13' )["Adj Close"] # daily Adj Close prices
prices_tech = yf.download(tech,  start='2022-01-01', end ='2024-04-13')["Adj Close"] # daily Adj Close prices
prices_snp = yf.download("^GSPC", start='2022-01-01', end ='2024-04-13')["Adj Close"]
prices_rf = yf.download("^IRX", start='2022-01-01', end ='2024-04-13')["Adj Close"] # risk free rate, 13 weeks treasury bills

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
# calculate returns
rets_energy = prices_energy.pct_change().dropna()
rets_tech = prices_tech.pct_change().dropna()
rets_snp = prices_snp.pct_change().dropna()
rets_rf = prices_rf.pct_change().dropna()

In [7]:
# add data energy
basecols_energy  = rets_energy.columns
rets_energy["^GSPC"] = rets_snp
rets_energy["RF"] = rets_rf

for name in rets_energy.columns.drop("RF"):
    rets_energy[name+"rf"] = rets_energy[name] - rets_energy["RF"] 
    
# add data tech
basecols_tech  = rets_tech.columns
rets_tech["^GSPC"] = rets_snp
rets_tech["RF"] = rets_rf

for name in rets_tech.columns.drop("RF"):
    rets_tech[name+"rf"] = rets_tech[name] - rets_tech["RF"] 
    
# add constant
rets_energy  = sm.add_constant(rets_energy)
rets_tech  = sm.add_constant(rets_tech)
    
rets_tech.head()

,const,CLSK,IBM,IOT,ORCL,PATH,QCOM,VTSI,^GSPC,RF,CLSKrf,IBMrf,IOTrf,ORCLrf,PATHrf,QCOMrf,VTSIrf,^GSPCrf
Date,,,,,,,,,,,,,,,,,,
2022-01-04,1.0,0.001043,0.014555,-0.064998,0.010694,-0.031457,0.005478,-0.042194,-0.000630,0.509434,-0.508391,-0.494879,-0.574432,-0.498740,-0.540890,-0.503956,-0.551628,-0.510064
2022-01-05,1.0,-0.107292,0.001449,-0.080703,-0.026789,-0.103318,-0.003899,-0.038179,-0.019393,0.062500,-0.169792,-0.061051,-0.143203,-0.089290,-0.165819,-0.066399,-0.100679,-0.081893
2022-01-06,1.0,-0.031505,-0.020836,-0.000869,0.002322,0.039633,-0.002949,0.048855,-0.000964,0.058824,-0.090329,-0.079660,-0.059693,-0.056502,-0.019191,-0.061773,-0.009969,-0.059787
2022-01-07,1.0,-0.019277,-0.003768,0.010004,0.013551,-0.010351,-0.029793,-0.034934,-0.004050,-0.022222,0.002945,0.018454,0.032227,0.035774,0.011871,-0.007571,-0.012712,0.018172
2022-01-10,1.0,-0.014742,0.001483,-0.009905,0.020226,-0.009694,-0.004046,0.004525,-0.001441,0.022727,-0.037469,-0.021244,-0.032633,-0.002501,-0.032421,-0.026774,-0.018202,-0.024168


In [8]:
#energy
modelcols_energy  = rets_energy.columns.where(rets_energy.columns.str.contains("rf")).dropna().drop("^GSPCrf")

for col in modelcols_energy:
    model=sm.OLS(rets_energy[col], rets_energy[['const','^GSPCrf']])  # Specifies the model
    results=model.fit()  # Estimates or fits the model
    print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  MPCrf   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     3771.
Date:                Mon, 15 Apr 2024   Prob (F-statistic):          3.20e-253
Time:                        11:39:40   Log-Likelihood:                 1446.0
No. Observations:                 571   AIC:                            -2888.
Df Residuals:                     569   BIC:                            -2879.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0018      0.001      2.166      0.0

<font color='crimson'>
    aphpa significant: 
    <br>
    alpha not significant: MPC, PSX, SUN, UGP, VLO
    <br>
    beta significant: MPC, PSX, SUN,  UGP, VLO
    <br>
    beta not significant:
    <br><br>
    Conclusion: CAPM holds for all!
</font>

In [10]:
#tech
modelcols_tech = rets_tech.columns.where(rets_tech.columns.str.contains("rf")).dropna().drop("^GSPCrf")

for col in modelcols_tech:
    model=sm.OLS(rets_tech[col], rets_tech[['const','^GSPCrf']])  # Specifies the model
    results=model.fit()  # Estimates or fits the model
    print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 CLSKrf   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     412.4
Date:                Mon, 15 Apr 2024   Prob (F-statistic):           2.31e-69
Time:                        11:40:04   Log-Likelihood:                 754.60
No. Observations:                 571   AIC:                            -1505.
Df Residuals:                     569   BIC:                            -1497.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0035      0.003      1.270      0.2

<font color='crimson'>
    aphpa significant: 
    <br>
    alpha not significant: CLS,  IBM, IOT, ORC,PATH,  QCOM,VTSI
    <br>
    beta significant: CLS, IBM,IOT, ORC, PATH,  QCOM,VTSI
    <br>
    beta not significant:
    <br><br>
    Conclusion: CAPM holds for all!
</font>

In [12]:
# re-estimate the models without intercept and get expected return with CAPM

# energy
eret_energy_list = []
for col in basecols_energy:
    eret_energy_list.append(erCAPM(rets_energy[col], rets_energy["^GSPC"], rets_energy["RF"]))
    
# tech
eret_tech_list = []
for col in basecols_tech:
    eret_tech_list.append(erCAPM(rets_tech[col], rets_tech["^GSPC"], rets_tech["RF"]))

eret_energy = pd.DataFrame(eret_energy_list, index =basecols_energy, columns = ["ER"])
eret_tech = pd.DataFrame(eret_tech_list, index =basecols_tech,  columns = ["ER"])

In [13]:
eret_energy

,ER
Ticker,
MPC,0.154656
PSX,0.168568
SUN,0.153139
UGP,0.110805
VLO,0.175979


In [14]:
eret_tech

,ER
Ticker,
CLSK,-0.070939
IBM,0.103916
IOT,-0.001285
ORCL,0.062144
PATH,-0.072393
QCOM,-0.031624
VTSI,0.100614


In [15]:
# weights for energy
# can change desired expected return

inv.targetP(ret=rets_energy[basecols_energy],mi=0.1,Bounds = [0,1],er_assumed=eret_energy["ER"])

{'w': array([3.86537744e-18, 1.06084097e-13, 2.90940057e-14, 1.00000000e+00,
        4.17630601e-19]),
 'er': 0.1,
 'vol': 0.4463775364571848}

In [16]:
# weights for tech
# can change desired expected return

inv.targetP(ret=rets_tech[basecols_tech],mi=0.15, Bounds=[0,1], er_assumed=eret_tech["ER"])

{'w': array([2.51846421e-19, 1.00000000e+00, 1.64523059e-16, 2.18715438e-13,
        3.74245051e-13, 2.48460000e-17, 9.11467046e-14]),
 'er': 0.15,
 'vol': 0.21092299297503747}

In [17]:
# PCA tech
PCA_exp_ret_tech=erPCA(rets_tech[basecols_tech],rets_tech["RF"])  #Expected returns estimationl using PCA method
PCA_exp_ret_tech

[3.0377490308845028,
 2.3248978791595,
 2.511976294917787,
 2.346979787996739,
 2.5586090506378247,
 2.403115457359172,
 2.3545166177743924]

In [18]:
# PCA weights for tech
# can change desired expected return

inv.targetP(ret=rets_tech[basecols_tech],mi=0.15, Bounds=[0,1], er_assumed=PCA_exp_ret_tech)

{'w': array([2.09973403e-17, 1.00000000e+00, 6.12621972e-13, 7.24249128e-17,
        6.10353568e-17, 1.55730058e-17, 3.78672041e-16]),
 'er': 0.15,
 'vol': 0.21092299297501574}

In [19]:
# PCA energy
PCA_exp_ret_energy=erPCA(rets_energy[basecols_energy],rets_energy["RF"])  #Expected returns estimationl using PCA method
PCA_exp_ret_energy

[2.6808674168368123,
 2.6762902058474345,
 2.4345295427527827,
 2.6762538691013775,
 2.762968130392645]

In [20]:
# PCA weights for energy
# can change desired expected return
inv.targetP(ret=rets_energy[basecols_energy],mi=0.1, Bounds=[0,1], er_assumed=PCA_exp_ret_energy)

{'w': array([2.13031685e-17, 5.15699089e-17, 1.00000000e+00, 5.78958120e-17,
        4.13792567e-17]),
 'er': 0.1,
 'vol': 0.23677703349990575}

<font color='DarkViolet ' style="font-size:15px"><b>CAPM and Factor Investing do not give great results!</b></font>